In [20]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm.session import sessionmaker
from sqlalchemy import inspect
from sqlalchemy.sql import func
import geoalchemy2
from sqlalchemy import Table, MetaData
from sqlalchemy.ext.declarative import declarative_base

In [16]:
def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(
    user=POSTGRES_USER,
    pw=POSTGRES_PW,
    url=POSTGRES_URL,
    db=POSTGRES_DB
)

In [6]:
engine = create_engine(DB_URL)

## Make a query 

In [7]:
Session = sessionmaker(bind=engine)
session = Session()

In [18]:
# Pure SQL command
res = session.query("""SUM(length_km) FROM se4all.distribution_line_se4all;""")
res.all()

In [25]:
# Same command in SQlAlchemy
metadata = MetaData(schema='se4all', bind=engine)

Base = declarative_base(metadata=metadata)

class DlinesSe4all(Base):
    __table__ = Table('distribution_line_se4all', Base.metadata, autoload=True, autoload_with=engine)
    
res = session.query(func.sum(DlinesSe4all.length_km))
res.all()

[(Decimal('16409.268'))]

## Look at the database

In [27]:
inspector = inspect(engine)
inspector.get_schema_names()

['geonode', 'import', 'information_schema', 'public', 'se4all']

In [8]:
inspector.get_table_names('se4all')

['data_version',
 'database_log',
 'cluster_offgrid',
 'boundary_adm1',
 'distribution_line_kaduna_electric',
 'cluster_all',
 'distribution_line_kedco',
 'osm_building',
 'distribution_line_se4all',
 'boundary_adm1_status']

In [28]:
inspector.get_columns('distribution_line_se4all', 'se4all')

[{'name': 'source',
  'type': VARCHAR(length=50),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'distribution_line_se4all_id',
  'type': INTEGER(),
  'nullable': False,
  'default': "nextval('se4all.distribution_line_se4all_distribution_line_se4all_id_seq'::regclass)",
  'autoincrement': True,
  'comment': None},
 {'name': 'source_distribution_line_se4all_id',
  'type': VARCHAR(length=50),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'name',
  'type': VARCHAR(length=500),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'description',
  'type': VARCHAR(length=500),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'condition',
  'type': VARCHAR(length=10),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'voltage_kv',
  'type': INTEGER(),
  'nullable': T